In [1]:
import os
import cv2
from tensorflow.keras.preprocessing.image import array_to_img
from tensorflow.keras.callbacks import Callback
import warnings
warnings.filterwarnings('ignore')

In [2]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from keras import Sequential, layers
from keras.layers import Conv2D, Dense, Flatten, Reshape, LeakyReLU, Dropout, UpSampling2D
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt 



In [ ]:
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import image_dataset_from_directory

In [ ]:

image_size = (128,128)
batch_size=64
directory='/kaggle/input/stanford-dogs-dataset/images/Images'



def get_ds(path):
  img = tf.io.read_file(path)
  img = tf.image.decode_jpeg(img,channels=3)
  img = tf.image.convert_image_dtype(img,tf.float32)
  img = tf.divide(tf.subtract(tf.multiply(img,255),127.5),127.5)
  return tf.image.resize(img,image_size) 

images=[]
for i in os.scandir(directory):
    for j in os.scandir(i.path):
        images.append(j.path)
        
images=tf.data.Dataset.from_tensor_slices(images)
train_images=images.map(get_ds,num_parallel_calls=tf.data.experimental.AUTOTUNE).batch(batch_size).shuffle(60000)

In [ ]:
def build_generator():
    model = tf.keras.Sequential()
    model.add(layers.Dense(8*8*128, use_bias=False, input_shape=(100,)))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    model.add(layers.Reshape((8, 8, 128)))
    assert model.output_shape == (None, 8, 8, 128)  # Note: None is the batch size

    model.add(layers.Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same', use_bias=False))
    assert model.output_shape == (None, 8, 8, 128)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(128, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    assert model.output_shape == (None, 16, 16, 128)
    
    model.add(layers.Conv2DTranspose(128, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    assert model.output_shape == (None, 32, 32, 128)
              
    model.add(layers.Conv2DTranspose(128, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    assert model.output_shape == (None, 64, 64, 128)
    
#     model.add(layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False))
#     model.add(layers.BatchNormalization())
#     model.add(layers.LeakyReLU())
#     assert model.output_shape == (None, 128, 128, 128)
    
    model.add(layers.Conv2DTranspose(3, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh'))
    print(model.output_shape)
    
    return model

In [ ]:
generator = build_generator()
generator.summary()

In [ ]:
noise = tf.random.normal([1,100])
generated_image = generator(noise,training=False)
plt.imshow(generated_image[0]*127.5+127.5)

In [ ]:
def build_discriminator(): 
    model = tf.keras.Sequential()
    model.add(layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same',
                                     input_shape=[128, 128, 3]))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))
    
    model.add(layers.Conv2D(256, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Flatten())
    model.add(layers.Dense(1))

    return model

In [ ]:
discriminator = build_discriminator()
discriminator.summary()

In [ ]:
print(discriminator(generated_image))

In [ ]:
# Adam is going to be the optimizer for both
from tensorflow.keras.optimizers import Adam
# Binary cross entropy is going to be the loss for both 
from tensorflow.keras.losses import BinaryCrossentropy

In [ ]:
cross_entropy=BinaryCrossentropy(from_logits=True)

In [ ]:
def discriminator_loss(real_output,fake_output):
  real_loss = cross_entropy(tf.ones_like(real_output),real_output)
  fake_loss = cross_entropy(tf.zeros_like(fake_output),fake_output)
  total_loss = real_loss + fake_loss
  return total_loss
     

In [ ]:
def generator_loss(fake_output):
  return cross_entropy(tf.ones_like(fake_output),fake_output)

In [ ]:
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

In [ ]:
checkpoint_dir = '/kaggle/working/training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir,'ckpt')
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)
manager = tf.train.CheckpointManager(checkpoint,checkpoint_dir,max_to_keep=3)

In [ ]:
EPOCHS = 500
noise_dims = 100
num_egs_to_generate = 16
seed = tf.random.normal([num_egs_to_generate,noise_dims])

In [ ]:
@tf.function
def train_step(images):
    
    noise=tf.random.normal([batch_size,noise_dims])
    
    with tf.GradientTape() as gen_tape, tf.GradientTape() as dis_tape:
        generated_images=generator(noise,training=True)
        
        real_output=discriminator(images,training=True)
        fake_output=discriminator(generated_images,training=True)
        
        gen_loss=generator_loss(fake_output)
        disc_loss=discriminator_loss(real_output,fake_output)
            
    gen_gradients=gen_tape.gradient(gen_loss,generator.trainable_variables)
    dis_gradients=dis_tape.gradient(disc_loss,discriminator.trainable_variables)
    
    generator_optimizer.apply_gradients(zip(gen_gradients,generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(dis_gradients,discriminator.trainable_variables))
    
    return gen_loss,disc_loss

In [4]:
from IPython import display
import time

total_gloss=[]
total_dloss=[]
def train(dataset,epochs):
    
    checkpoint.restore(manager.latest_checkpoint)
    if manager.latest_checkpoint:
        print(f'Restored from {manager.latest_checkpoint}')
    else:
        print('Initializing from scratch')
    
    for epoch in range(epochs):
        disc_loss=gen_loss=0
        start=time.time()
        count=0
        for batch in dataset:
            losses=train_step(batch)
            count+=1
            disc_loss+=losses[1]
            gen_loss+=losses[0]
            
        
        total_gloss.append(gen_loss.numpy())
        total_dloss.append(disc_loss.numpy())
        
            
        if (epoch+1)%50==0:
            checkpoint.save(file_prefix=checkpoint_prefix)
            
#             display.clear_output(wait=True)
            generate_and_save_output(generator,epoch+1,seed)
            
        print(f'Time for epoch {epoch + 1} is {time.time()-start}')
        print(f'Gloss: {gen_loss.numpy()/count} , Dloss: {disc_loss.numpy()/count}',end='\n\n')
        
#     display.clear_output(wait=True)
    generate_and_save_output(generator,epochs,seed)

In [3]:
def generate_and_save_output(model,epoch,test_input):

      predictions = model(test_input,training=False)
      # predictions = predictions.numpy().reshape(16,64,64,1)
      fig = plt.figure(figsize=(4,4))
      # print(predictions)
      for i in range(predictions.shape[0]):
        plt.subplot(4,4,i+1)
        plt.imshow((predictions[i]*127.5+127.5).numpy().astype(np.uint8),cmap='gray')
        plt.axis('off')
      plt.savefig(f'/kaggle/working/image_at_epoch_{epoch}.png')
      plt.show()

In [ ]:
train(train_images,EPOCHS)

In [ ]:
new_image = generator(tf.random.normal([1,100]),training=False)
plt.imshow((new_image[0]*127.5+127.5).numpy().astype(np.uint8))

In [ ]:
generator.save('/kaggle/working/generator.keras')